# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [67]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heatmap_layer = gmaps.heatmap_layer(
    weather_df[['Lat', 'Lng']], weights=weather_df['Humidity'],
    max_intensity=50, point_radius=5.0
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
ideal_weather_df = weather_df[(weather_df['Max Temp'] >= 70) & 
                              (weather_df['Max Temp'] <= 80) & 
                              (weather_df['Wind Speed'] <= 10) &
                              (weather_df['Cloudiness'] == 0)]
ideal_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
46,51,abu dhabi,0.0,AE,1.584493e+09,64.0,24.47,54.37,75.00,3.36
78,86,airai,0.0,TL,1.584493e+09,67.0,-8.93,125.41,72.90,4.52
137,149,antalaha,0.0,MG,1.584493e+09,84.0,-14.90,50.28,78.30,4.94
168,181,emilio carranza,0.0,MX,1.584493e+09,77.0,19.97,-96.61,76.39,9.98
257,279,jati,0.0,PK,1.584493e+09,24.0,24.35,68.27,72.03,7.11
271,294,morondava,0.0,MG,1.584493e+09,93.0,-20.28,44.28,71.76,3.40
442,482,dwarka,0.0,IN,1.584493e+09,61.0,22.24,68.97,78.19,8.63
506,554,aloleng,0.0,PH,1.584493e+09,60.0,16.13,119.78,79.27,9.57


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = ideal_weather_df
hotel_df['Hotel Name'] = ""

C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [54]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "types": "lodging",
    "rankby": "distance",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    location = f"{str(row['Lat'])}, {str(row['Lng'])}"

    # add keyword to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {location}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {location} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 46: 24.47, 54.37.
Closest hotel in 24.47, 54.37 is Marriott Executive Apartments Downtown Abu Dhabi.
------------
Retrieving Results for Index 78: -8.93, 125.41.
Closest hotel in -8.93, 125.41 is Juvinal Martin.
------------
Retrieving Results for Index 137: -14.9, 50.28.
Closest hotel in -14.9, 50.28 is Hotel Vitasoa.
------------
Retrieving Results for Index 168: 19.97, -96.61.
Closest hotel in 19.97, -96.61 is hotel delicias.
------------
Retrieving Results for Index 257: 24.35, 68.27.
Closest hotel in 24.35, 68.27 is Muhammad Khuzaima Traders.
------------
Retrieving Results for Index 271: -20.28, 44.28.
Closest hotel in -20.28, 44.28 is eden rock hotel.
------------
Retrieving Results for Index 442: 22.24, 68.97.
Closest hotel in 22.24, 68.97 is Hotel Rajmandir.
------------
Retrieving Results for Index 506: 16.13, 119.78.
Closest hotel in 16.13, 119.78 is Agno Umbrella Rocks.
------------


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [62]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…